In [ ]:
# Connecting to Google Drive to Save the notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Installing dependencies
!pip install transformers sentencepiece

In [37]:
# importing libraries
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import BertForSequenceClassification
from transformers import WhisperTokenizer
from IPython.display import Audio, display
import numpy as np

In [13]:
# Always check the documentation fo the model to see how it can be used
# The model can be used with transformers.pipeline
# The task provided in the pipeline will download a default model
# The model will be downloaded and used (that is the reason for using Google Collab)

# Storing the task to be performed
task = "sentiment-analysis"

classifier_sentiment = pipeline(task)

output = classifier_sentiment("I'm happy to be learning about hugging face hub")

print(output)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998220801353455}]


In [10]:
# With the model already stored new values can be sent
output = classifier_sentiment("I was not really happy about the things you said to me yesterday")

print(output)

[{'label': 'NEGATIVE', 'score': 0.9997748732566833}]


In [14]:
# Not all models support every language. Search a multilanguage model or select it by language
# The model can also be loaded directly

# Selecting multilanguage model
language_model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"

# Tokenizer of language model for preprocessing text
tokenizer_model = AutoTokenizer.from_pretrained(language_model_name)

# Instancing pretrained model
language_model = AutoModelForSequenceClassification.from_pretrained(language_model_name)

# Creating the pipeline to execute the task
classifier_sentiment = pipeline(task = task, model=language_model, tokenizer=tokenizer_model)

# Using the model (In PT-BR)
output = classifier("Não fiquei feliz com a forma que fui tratado ontem!")

print(output)


[{'label': 'NEGATIVE', 'score': 0.8799389004707336}]


In [15]:
output = classifier("Eu amei assistir aquele filme!")

print(output)

[{'label': 'POSITIVE', 'score': 0.9880164861679077}]


In [33]:
# Sentiment analysis for finance (in PT-BR)

# List of phrases that will be considered
list_phrases = ["A bolsa subiu hoje",
                "Petrobras está um desastre"] #"The stock market went up today.", "Petrobras is a disaster"

# Setting the prediction map
prediction_map = {
    0: "Positivo", # Positive
    1: "Neutro",   # Neutral
    2: "Negative" # Negative
}

# Model name
language_model_finan_name = "lucas-leme/FinBERT-PT-BR"

# Tokenizer of language model for preprocessing text
tokenizer_finan_model = AutoTokenizer.from_pretrained(language_model_finan_name)

# Tokenizer for prediction map
tokens = tokenizer_finan_model(list_phrases,
                               return_tensors = "pt",
                               padding = True,
                               truncation = True,
                               max_length = 512)

# Instancing pretrained model
finan_language_model = BertForSequenceClassification.from_pretrained(language_model_finan_name)

# Storing the outputs of the listed phrases from the model
finan_model_outputs = finan_language_model(**tokens)

# The model returns several outputs. The numerica values can be found in finan_model_outputs.logits
# Storing the results for each of the phrases
finan_output = [prediction_map[np.argmax(finan)] for finan in finan_model_outputs.logits.cpu().detach().numpy()]

for i in range(0, len(list_phrases)):
  print(f"{list_phrases[i]} - {finan_output[i]}")




A bolsa subiu hoje - Positivo
Petrobras está um desastre - Neutro


In [36]:
# Using a model to translate an audio file to text

whisper_model_name = "openai/whisper-small"

task = "automatic-speech-recognition"

whisper = pipeline(task = task, model = whisper_model_name)


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [42]:
display(Audio("/content/drive/MyDrive/Colab Notebooks/HuggingFaceHub - Data/letsdata.mp3"))

In [45]:
# Getting the audio text

transcription = whisper("/content/drive/MyDrive/Colab Notebooks/HuggingFaceHub - Data/letsdata.mp3")

print(transcription)

{'text': " Boa noite! Sejam todos bem-vindos a mais uma aula de aprendizagem de máquina. Hoje veremos Hug in Face Hub, como funciona o supermercado de modelos. Tudo aqui no Let's Data."}


In [47]:
# Translating the audio file

# Instancing the model for the current audio file
whisper_tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-medium", language="pt", task="transcribe")

# Decoding for output language
forced_decoder_ids = whisper_tokenizer.get_decoder_prompt_ids(language="en", task="translate")

# Translating
transcription = whisper("/content/drive/MyDrive/Colab Notebooks/HuggingFaceHub - Data/letsdata.mp3", generate_kwargs={"forced_decoder_ids": forced_decoder_ids})

print(transcription)



tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'text': " Good night and welcome to another lesson of machine learning. Today we will see how the super market of models works, all here at Let's Data."}


In [48]:
# Using image to text conversion

task = "image-to-text"

img_text_model = "nlpconnect/vit-gpt2-image-captioning"

img_text = pipeline(task = task, model = img_text_model)

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [49]:
img_text("https://s2-ge.glbimg.com/fkgUkJr09cdT4jUyiX5RtF0GAVI=/0x0:5103x3197/924x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_bc8228b6673f488aa253bbcb03c80ec5/internal_photos/bs/2019/q/j/pzfGVtQdipIUJKeNgfNw/amanda.jpg")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


[{'generated_text': 'two boys playing soccer on a field '}]

In [50]:
img_text("https://img.freepik.com/fotos-gratis/retrato-de-um-cute-menina-comer-maca-vermelha-madura_23-2147893285.jpg?w=2000")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'a little girl eating a piece of fruit '}]